# Script for Generating Training Data from Frames using PIPs
https://github.com/aharley/pips

## Initial set up
Connection to MyDrive, requirements, imports, downloading models weights

In [2]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# installing requirements
!pip install imageio==2.19.3
!pip install opencv-python==4.6.0.66
!pip install einops==0.4.1
!pip install scikit-learn==1.1.1
!pip install matplotlib==3.5.1
!pip install protobuf==3.20.0
!pip install fire==0.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 15.3 MB/s 
     |████████████████████████████████| 3.2 MB 57.7 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.2 MB 1.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.3 MB 14.8 MB/s 
     |████████████████████████████████| 965 kB 71.8 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 15.1 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.57.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-translate 3.8.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 6.2 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=3b07f63c9a0e679f7678729895268d41c59347eeb5ddc513c650b64950c70908
  Stored in directory: /root/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire


In [2]:
import sys
sys.path.append('/content/drive/My Drive/drive_folder/pips')
import time
import numpy as np
import io
import os
from PIL import Image
import cv2
import saverloader
from nets.pips import Pips
import utils.improc
import random
import glob
from utils.basic import print_, print_stats
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gc
import imageio.v2 as imageio
from csv import writer

# save numpy array as npz file
from numpy import asarray
from numpy import savez_compressed

In [3]:
# downloading their reference model
!bash /content/drive/MyDrive/drive_folder/pips/get_reference_model.sh

downloading the model from dropbox...
--2023-01-01 10:47:37--  https://www.dropbox.com/s/hbo7ns4vfx1sejp/reference_model.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/hbo7ns4vfx1sejp/reference_model.tar.gz [following]
--2023-01-01 10:47:37--  https://www.dropbox.com/s/raw/hbo7ns4vfx1sejp/reference_model.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc654b015fc500ae7887c23192eb.dl.dropboxusercontent.com/cd/0/inline/BzuP7_gOyCGuv_jZ2EZNvozamWOOaRN_b5Loz6fLwj2qmTjRYpihm5y-ZMb2S-4IBCX0zkdjvz_N6iZwVs0kIDUlmwPNe1ueNanXBH0AWDkntFlo5FCkMdDRYdMKHmmUWmx0Yj361TwX42jJhqTKyBJ8iRjve6lfJOTsk5CeuBV_LQ/file# [following]
--2023-01-01 10:47:38--  https://uc654b015fc500ae7887c23192eb.dl.dropboxusercontent.com/cd/0/inline/BzuP7_gOy

## Loading directory with data frames to generate training data from

In [ ]:
DATA_DIR = "frames" # specify the name of the data folder located in drive
!unzip -d "$DATA_DIR"/ /content/drive/MyDrive/"$DATA_DIR".zip # unziping frames

## Generating training data

### PIPs implementation

In [9]:
def create_model(model_weights):
  model = Pips(stride=4).cuda()
  parameters = list(model.parameters())
  _ = saverloader.load(model_weights, model)
  global_step = 0
  model.eval()
  return model


def run_model(model, rgbs, N, split):
    rgbs = rgbs.cuda().float() # B, S, C, H, W

    B, S, C, H, W = rgbs.shape
    rgbs_ = rgbs.reshape(B*S, C, H, W)
    H_, W_ = 360, 640
    rgbs_ = F.interpolate(rgbs_, (H_, W_), mode='bilinear')
    H, W = H_, W_
    rgbs = rgbs_.reshape(B, S, C, H, W)

    # pick N points to track; we'll use a uniform grid
    N_ = np.sqrt(N).round().astype(np.int32)
    grid_y, grid_x = utils.basic.meshgrid2d(B, N_, N_, stack=False, norm=False, device='cuda')
    grid_y = 8 + grid_y.reshape(B, -1)/float(N_-1) * (H-16)
    grid_x = 8 + grid_x.reshape(B, -1)/float(N_-1) * (W-16)
    xy = torch.stack([grid_x, grid_y], dim=-1) # B, N_*N_, 2
    _, S, C, H, W = rgbs.shape

    print_stats('rgbs', rgbs)

    # splitting grid of points into tensors of size 1500 along dim=1
    if split:
       xy_split = torch.split(xy, 1500, dim=1)
       preds_split = []
       vis_split = []

      # for each splitted point compute trajs
       for i in range(len(xy_split)):
         preds, preds_anim, vis, stats = model(xy_split[i], rgbs, iters=6)
         preds_split.append(preds[-1])
         vis_split.append(vis[-1])

       # put trajs back together
       trajs_e = torch.cat(preds_split, 2)
       vis_e = torch.cat(vis_split, 1).unsqueeze(0)
       

    else:
      preds, preds_anim, vis, stats = model(xy, rgbs, iters=6)
      # preds is a list of torch tensors 
      trajs_e = preds[-1] # tensor of shape (frames, tracking points, 2)
      vis_e = vis

    print_stats('vis_e', vis_e)
    return trajs_e[:,-1,:,:], vis_e[:,-1,:]

def generate_training_data(pips_model, video_name, n, split=True):
    model = pips_model
    
    ## Choose hyps
    B = 1
    S = 8
    N = n**2 # number of points to track

    # Get frames of a file
    filenames = glob.glob(DATA_DIR+'/content/frames/'+video_name+'/frames/*.jpg')
    filenames = sorted(filenames)

    # Remove every second frame from a list
    n = 2
    del filenames[n - 1::n]

    max_iters = len(filenames)//S # run each unique subsequence
    iters = 2
    global_step = 0

    # Run model each of 8 frames
    while global_step < iters:
        global_step += 1
        
        try:
            rgbs = []
            sample_id = video_name[-11:]+"_"+str((global_step-1)*S)
            print("sample {}: step {}/{}".format(sample_id, global_step, max_iters))

            # skip generating this sample if already in the log list
            if sample_id in LOG_LIST:
                  print(sample_id+" already in log list")
                  continue
            for s in range(S):
                frame_num = (global_step-1)*S+s
                fn = filenames[frame_num]
                if s==0:
                    print('start frame', fn)
                im = imageio.imread(fn)
                im = im.astype(np.uint8)
                im = torch.from_numpy(im).permute(2,0,1)
                rgbs.append(im)
            rgbs = torch.stack(rgbs, dim=0).unsqueeze(0) # 1, S, C, H, W

            with torch.no_grad():
                trajs_e, vis_e = run_model(model, rgbs, N, split)

            # store img0 and img1 (frame 1 and 8) and trajs_e, vis_e
            print(vis_e)
            save_data(sample_id, rgbs[0][0], rgbs[0][-1], trajs_e, vis_e)

        except FileNotFoundError as e:
            print('error', e)

def img_to_array(tensor_img):
   x = tensor_img.permute(1, 2, 0).numpy() # from tensor to numpy array
   x = cv2.resize(x, (240, 120)) # resizing image
   return x

def save_data(sample_id, frame0, frame7, trajs, vis):
  """
  Saving
  - coordinates to COORDS_DIR as .npy files
  - visibility of coordinates to VIS_DIR as .npy files
  - 0th frames to FRAME0_DIR directory as .npy files
  - 7th frames to FRAME7_DIR directory as .npy files
  
  - Coordinates, visibility and frames are mapped with sample_id, all sample_ids are stored in sample_ids.txt
  """
  img0 = img_to_array(frame0) # transform first frame to array
  img7 = img_to_array(frame7) # tranform last frame (8th) to array
  coords = trajs.cpu().numpy()
  vis = vis.cpu().numpy()

  # Store coords, frame0, frame1
  np.save(COORDS_DIR+sample_id, coords)
  np.save(VIS_DIR+sample_id, vis)
  np.save(FRAME0_DIR+sample_id, img0)
  np.save(FRAME1_DIR+sample_id, img7)

  # Add sample_id to log list
  LOG_LIST.append(sample_id)
  save_log(LOG_LIST)

def save_log(log_list):
  with open(LOG_FILE, "w") as outfile:
    outfile.write("\n".join(log_list))

In [6]:
model = create_model('reference_model')

reading ckpt from reference_model
...found checkpoint reference_model/model-000100000.pth


### Creating directories and running generation of data

In [7]:
# create a folder to store training data in
!mkdir training_data
!mkdir training_data/frame0/
!mkdir training_data/frame1/
!mkdir training_data/coords/
!mkdir training_data/vis/
!touch training_data/sample_ids.txt
PATH = ""

In [ ]:
# if we want to use an old training data folder
# cp from drive to colab memory and unzip
TRAIN_DATA_DIR = "training_data_14122022" # specify the name of the data folder located in drive
!unzip -d "$TRAIN_DATA_DIR"/ /content/drive/MyDrive/"$TRAIN_DATA_DIR".zip # unziping frames
PATH = TRAIN_DATA_DIR+"/content/"

In [8]:
TRANING_DATA_DIR = PATH+"training_data/"
COORDS_DIR = PATH+"training_data/coords/"
VIS_DIR = PATH+"training_data/vis/"
FRAME0_DIR= PATH+"training_data/frame0/"
FRAME1_DIR= PATH+"training_data/frame1/"
LOG_FILE = PATH+"training_data/sample_ids.txt"
LOG_LIST = open(LOG_FILE).read().splitlines()

In [10]:
generate_training_data(model, 'Tai chi10 forms 2009-KX-dEeB47sc', n=64)
# generate_training_data(model, 'Tai chi陈式太极拳五十六式-vB8XTJfV4rY', n=64)
# generate_training_data(model, 'Tai chiYang Family Tai Chi q&a Knee Brush-WPeVwAhTNuU', n=64)
# generate_training_data(model, 'Tai chiWee Kee Jin - Keeping Your Structure-7jn9jeAbChE', n=64)
# generate_training_data(model, 'Tai chi熊門楊家太極拳111式第一段\u3000李國光老師示範-X_9SJZuSWQU', n=64)
# generate_training_data(model, 'Tai chi56式夕陽美功夫扇-uOw-z7CR7x8', n=64)
# generate_training_data(model, 'Tai chiRen Guang Yi - Chen Style Taiji silk reeling part II-Wh3AmDsdQtM', n=64)

sample KX-dEeB47sc_0: step 1/235
start frame frames/content/frames/Tai chi10 forms 2009-KX-dEeB47sc/frames/frame_00001.jpg
rgbs (float32) min = 0.00, mean = 0.59, max = 199.60 torch.Size([1, 8, 3, 360, 640])
vis_e (float32) min = -4.14, mean = 3.00, max = 11.26 torch.Size([1, 8, 4096])
tensor([[ 2.3131, -0.0137,  1.5245,  ..., -0.7300, -4.1350, -3.1928]],
       device='cuda:0')
sample KX-dEeB47sc_8: step 2/235
start frame frames/content/frames/Tai chi10 forms 2009-KX-dEeB47sc/frames/frame_00017.jpg
rgbs (float32) min = 0.00, mean = 16.16, max = 217.67 torch.Size([1, 8, 3, 360, 640])
vis_e (float32) min = -4.80, mean = 2.77, max = 12.48 torch.Size([1, 8, 4096])
tensor([[ 0.7453,  1.7871,  0.2403,  ...,  1.1205, -0.0788, -1.1653]],
       device='cuda:0')


In [ ]:
# create a list of file names to go through
videos_list = [f for f in os.listdir(DATA_DIR+'/content/frames/') if not f.startswith('.')]
print(videos_list)

In [ ]:
# iterate through the videos list and generate training data files
for x in range(len(videos_list)):
  print("Now generating training data for {}, {} out of {}.".format(videos_list[x], x, len(videos_list)))
  generate_training_data(model, videos_list[x], 100)

### Zip data and store in Drive

In [ ]:
from datetime import date
TODAY = date.today()

# zip training data
!cd /content/"$PATH"training_data && zip -r /content/training_data_"$TODAY".zip .
# cp NumPy zip file into drive
!cp training_data_"$TODAY".zip /content/drive/MyDrive/